# PROJET TVID

In [13]:
import numpy as np
from PIL import Image

## A. Jouer un flux MPEG-2 élémentaire de test

### 1. Dans le dossier videos/elementary, avec vlc, visualisez les séquences MPEG-2 suivantes : 
### _vlc -V \<renderer> bw_numbers.m2v / pendulum.m2v / Jaggies1.m2v_

Les commandes utilisees sont :
- vlc -V x11 bw_numbers.m2v
- vlc -V x11 pendulum.m2v
- vlc -V x11 Jaggies1.m2v

 <img src="./screenshots/1.png" title="bw_numbers">
 <img src="./screenshots/2.png" title="pendulum">
 <img src="./screenshots/3.png" title="Jaggles1">

### 2. Avec mpeg2dec, convertissez en pile d’images votre séquence MPEG-2 choisie (cf. aide de mpeg2dec).

Nous avons cree un dossier pgm_images a la racine du projet, et avons effectue cette commande :
- _../tools/mpeg2dec/src/mpeg2dec -o pgm ../videos/elementary/bw_numbers.m2v_

Pour convertir la sequence bw_numbers.m2v en pile d'images.

 <img src="./screenshots/pgm_0.png" title="0">

### 3. Observez les pgm générées. Comment sont-elles structurées? Quel est le format de l’image: résolution, profondeur, sampling mode?

In [11]:
img = Image.open('./pgm_images/0.pgm')

print(f"Format: {img.format}")
print(f"Resolution: {img.size}")
print(f"Mode: {img.mode}")
print(f"Color palette: {img.palette}")

Format: PPM
Resolution: (720, 720)
Mode: L
Color palette: None


Nos images sont au format PGM. Ce sont des images en niveaux de gris (sur 8-bit). Elles ont une resolution de 720x720.

 <img src="./screenshots/bw_numbers_infos.png" title="bw_numbers_infos">
 
En regardant les informations de la video originale, nous pouvons voir que la resolution est de 720x480, que la profondeur est de 8-bit, et que le sampling mode est en 4:2:0 YUV.

Les frames sont composes de 2 images consécutives entrelacées dans une même image (top first field).

Les informations de la video ont ete generees par cette commande :
- ffmpeg -i bw_numbers.m2v

### 4. Modifiez mpeg2dec pour logger simplement les flags progressive_frame, top_field_first, repeat_first_field de chaque image décodée.

Pour logger simplement les flags progressive_frame (PROG), top_field_first (TFF) et repeat_first_field (RFF) pour chaque image decodee, il faut modifier le fichier :
- mpeg2dec/src/mpeg2dec.c, ligne 55

en mettant la variable __verbose__ a 1 :
- static int verbose = 1;

On obtient par exemple ces logs pour la video bw_numbers.m2v :
 <img src="./screenshots/bw_numbers_logs.png" title="bw_numbers_logs">
 
Les images decodee ici sont toutes TFF.

### 5. Avec votre propre code et dans le langage de votre choix, implémentez un convertisseur d’images vers un format plus humainement lisible (ppm RGB est assez universel).

In [15]:
def yuv_to_rgb(y, u, v):
    """
    Convert YUV pixel to RGB.
    """
    r = y + 1.402 * (v - 128)
    g = y - 0.34414 * (u - 128) - 0.71414 * (v - 128)
    b = y + 1.772 * (u - 128)
    return np.clip([r, g, b], 0, 255)

def convert_pgm_to_ppm(pgm_path, ppm_path):
    """
    Convert a PGM image (assumed to be in YUV format) to PPM (RGB format).
    """
    # Load the PGM image
    with Image.open(pgm_path) as img:
        yuv_img = np.array(img)

    # Assuming the YUV image is in YUV444 format
    height, width = yuv_img.shape
    rgb_img = np.zeros((height, width, 3), dtype=np.uint8)

    # Convert each pixel
    for i in range(height):
        for j in range(width):
            y, u, v = yuv_img[i, j], yuv_img[i, j], yuv_img[i, j] # Assuming Y, U, V are the same in grayscale image
            rgb_img[i, j] = yuv_to_rgb(y, u, v)

    # Save the result
    Image.fromarray(rgb_img).save(ppm_path)

In [16]:
# Example usage
pgm_path = './pgm_images/0.pgm'  # Replace with the actual path to your PGM file
ppm_path = './ppm_images/0.ppm'  # Replace with the desired path for the PPM file
convert_pgm_to_ppm(pgm_path, ppm_path)

 <img src="./screenshots/pgm_0.png" title="pgm_0">
 <img src="./screenshots/ppm_0.png" title="ppm_0">

Les deux images (pgm au dessus et ppm en dessous) semblent identiques. 

L'image ppm, passant du format YUV au format RGB, se voit attribuer des couleurs supplementaires, chose normale puisque les cannaux UV donnant l'information de couleur, ne servant pas dans une image pgm en noir et blanc, contribue aux informations pour les cannaux RGB lors de la conversion.

La fonction de conversion semble donc fonctionner comme attendu.